In [1]:
from Bio.SeqIO.FastaIO import SimpleFastaParser
import os, sys
import pandas as pd
import pyarrow

# MPRAFlow

In [30]:
PREFIX = "null_mpra_head"
PATH = os.path.join("/wynton/home/ahituv/fongsl/nullomers/data/lock/20230815_association_sequencing",
                    PREFIX)

FILE = os.path.join(PATH, f"{PREFIX}_barcodes_per_candidate-no_repeats-no_jackpots.feather")


df = pd.read_feather(FILE)

df.head()

In [32]:
df["n_barcodes"].sum()  # mapq=30 23646  # mapq = 20 36403

36403

In [33]:
df.describe()

,n_barcodes
count,28200.000000
mean,1.290887
std,0.598497
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,8.000000


In [34]:
len(df['coord'].unique())

28200

In [35]:
df[["null_id", "peak_id", "activity"]] = df["coord"].str.split('|', expand=True)

df.head()

,coord,n_barcodes,null_id,peak_id,activity
0,15-firstorder.63414|WTC11_seq51996_F|inactive,1,15-firstorder.63414,WTC11_seq51996_F,inactive
1,15-firstorder.1307144|HepG2_DNasePeakNoPromote...,4,15-firstorder.1307144,HepG2_DNasePeakNoPromoter58688,active
2,15-firstorder.2355935|HepG2_DNasePeakNoPromote...,1,15-firstorder.2355935,HepG2_DNasePeakNoPromoter50640,inactive
3,15-firstorder.1412646|K562_peak57048|inactive,1,15-firstorder.1412646,K562_peak57048,inactive
4,15-firstorder.192228|WTC11_seq41747_F|inactive,3,15-firstorder.192228,WTC11_seq41747_F,inactive


In [36]:
df.loc[df["coord"].str.contains("C:SLEA"), "activity"] = "ctrl"

In [37]:
df.groupby('activity')["n_barcodes"].describe()

,count,mean,std,min,25%,50%,75%,max
activity,,,,,,,,
active,9192.0,1.249130,0.547344,0.0,1.0,1.0,1.0,7.0
ctrl,18.0,1.055556,0.416176,0.0,1.0,1.0,1.0,2.0
inactive,18534.0,1.311482,0.621509,0.0,1.0,1.0,1.0,8.0


# mapq=20
	count 	mean 	std 	min 	25% 	50% 	75% 	max
    activity 								
    active 	9192.0 	1.249130 	0.547344 	0.0 	1.0 	1.0 	1.0 	7.0
    ctrl 	18.0 	1.055556 	0.416176 	0.0 	1.0 	1.0 	1.0 	2.0
    inactive 	18534.0 	1.311482 	0.621509 	0.0 	1.0 	1.0 	1.0 	8.0

# mpaq = 30
    count 	mean 	std 	min 	25% 	50% 	75% 	max
    activity 								
    active 	6.0 	1.000000 	0.000000 	1.0 	1.0 	1.0 	1.0 	1.0
    ctrl 	18.0 	1.055556 	0.416176 	0.0 	1.0 	1.0 	1.0 	2.0
    inactive 	17631.0 	1.306052 	0.615297 	0.0 	1.0 	1.0 	1.0 	8.0


# trouble shooting
active elements - 
- decrease mapq
- make active-speific .fasta

Fastq PE split-
- increase number of lines to test split. 

# Sarah's python 

In [16]:
PATH = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc"
FILE = os.path.join(PATH, "null_mpra_barcodes_per_candidate-no_repeats-no_jackpots.feather")

df = pd.read_feather(FILE)

df.head()

,coord,n_barcodes
0,endog|HepG2_DNasePeakNoPromoter10535|active,10342951


# pickle

In [ ]:
PICKLE = pd.read_pickle(os.path.join(PATH, "null_mpra_coords_to_barcodes.pickle"))

In [17]:
p = pd.DataFrame(PICKLE.items())

In [32]:
p.columns =["bc", "n"]

In [33]:
p.head()

,bc,n
0,15-firstorder.63414|WTC11_seq51996_F|inactive,"[AAACATTAGTTTCTT, GAAGAGGGCCAAACC, GCAGTTGTGTA..."
1,15-firstorder.2355935|HepG2_DNasePeakNoPromote...,"[TTGGAATGACCAACA, TGACTGAGAAGGCTA, TACTTGGACAT..."
2,15-firstorder.1412646|K562_peak57048|inactive,"[TTTCTTTGGGATTAA, ATATTGCTCCCTGCG, TTAAGGAGATG..."
3,15-firstorder.192228|WTC11_seq41747_F|inactive,"[CTATGTTATTACAAA, GATCCTACATAGAGT, TACATTACGTG..."
4,15-firstorder.1426742|HepG2_DNasePeakNoPromote...,"[GAACTTGTCTGGCTA, CTATGAAAGAGATCC, CAAAACATAGT..."


In [34]:
len(set(p["bc"]))

56504

In [ ]:
# there are 10358215 bc

# FASTP run 

In [2]:
FP="/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc_fastp/null_mpra_coords_to_barcodes.pickle"

In [4]:
pickle = pd.read_pickle(FP)
p = pd.DataFrame(pickle.items())

In [5]:
p.shape

(1, 2)

In [6]:
p.head()

,0,1
0,endog|HepG2_DNasePeakNoPromoter10535|active,"[AAACATTAGTTTCTT, TTGGAATGACCAACA, TTTCTTTGGGA..."


In [3]:
a='TGATTCTTACTTCGTGCTCCAAGACGTCTGTTCTTTCCTGCTTTGTGACGCAGGCCTCCGTGCTGGCATTTGTGAAATGCAGATAACCTGTCAAACGTTGTCGACGGCCAGTACTCTGGTTGTAATCTCCTGAGGCCAGGAGCTGGGTCTTCAGGTCTTCACTTTCTCTCTATGTCCCCAACACAGCTCTGGGGGCGGGG'
len(a)

200